In [153]:
from bs4 import BeautifulSoup
import codecs
import re

f=codecs.open("files/combined/ap", 'r')
soup = BeautifulSoup(f)
text = soup.findAll('text') 

#removing tags
output = ''.join([tag.text.strip() for tag in text])

In [154]:
from nltk import sent_tokenize
sentences = sent_tokenize(output)

In [155]:
# function to preprocess text
def clean(text):
    # removing new line characters
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    text = re.sub("_",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    text = re.sub('\`','',str(text))
    text = re.sub('\'','',str(text))
    
    return text

# create a variable for the for loop results
sentencecleaned = []

#clean the sentences with the function
for a in sentences:
    sentencecleaned.append(clean(a))

In [12]:
#%store sentencecleaned
%store -r sentencecleaned

In [13]:
#extract all sentences containing ' rock '
examples1=[sentence for sentence in sentencecleaned if ' rock ' in sentence]
examples1=examples1[0:200] #subsetting

#%store examples1
#%store -r examples1

#print(examples1) #100 whole sentences

In [14]:
#extract 3 surrounding words (just the context)
import re
context1=[]

def contexts(examples,context, word):
    context=[]
    sub='(\w*)\W*(\w*)\W*(\w*)\W*('+word+')\W*(\w*)\W*(\w*)\W*(\w*)' #3 words before and after
    for a in examples:
        for i in re.findall(sub, a, re.I):
            context.append(" ".join([x for x in i if x != ""])) #adding the context examples into list
    return context

context1=contexts(examples1,context1,' rock ') #here the word should include an empty space before and after, to get only 'rock' words
#print(context1) 

In [15]:
import numpy as np

#classified 100 examples
#THIS ONE is with class 3 and a couple of extra classes. The class 3 will be deleted
class1 = np.array( [1,2,2,1,2,1,1,1,1,1,1,3,2,1,2,1,1,1,2,2,1,1,1,1,3,1,1,1,2,2,2,1,2,2,1,1,1,1,2,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,1,2,1,1,1,1,1,1,2,1,2,2,1,1,1,1,2,1,1,1,3,2,2,1,1,1,1,1,2,2,2,2,2,2,2,2,1] )
#shape = (10,10)
#class1.reshape( shape )


#
# IMPORTANT!!!!
#
# only run the cells which use the delete method ONLY ONCE
# 


del_index=[11,24,87]
class1=np.delete(class1,del_index)
print(class1)
#shape = (10,10)
#class3.reshape( shape )
context1=np.delete(context1,del_index)
print(len(class1))
print(len(context1))

[1 2 2 1 2 1 1 1 1 1 1 2 1 2 1 1 1 2 2 1 1 1 1 1 1 1 2 2 2 1 2 2 1 1 1 1 2
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 1 2 1 1 1 1 1 1 2
 1 2 2 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 2 2 2 2 2 2 2 2 1]
101
209


In [16]:
#If you have the corresponding variable class1
#and the context variable in the same form than context1
#Then you can use this method to obtain the collocalations
#class1 and context1 variables must be the same length
#The different senses of the targert word must be written in the class variable
# as starting from 0 and continuing from.

#collocalatsK gives you +- k collocalations 
#collocalatsPLUS1 gives you +1 collocalations
#collocalatsMINUS1 gives you -1 collocalations

def collocalatsK(clas, contex):
    
    #Making one big string of all the context sentences
    wordS=" ".join(contex)
    #Splitting senteces into words
    wordS=wordS.split(' ')
    #Making a set of those words
    word_SET=set(wordS)
    
    classes=int(max(clas))
    #Making a matrix of the length of the words in a set
    #and a width of the different senses of the target word
    collocalationS=np.zeros((len(word_SET),classes)) #MODIFIED to exclude +1
    
    #In the 1. for loop going through of all the words in a set
    #In the 2. for loop going through of all the contexts and checking
    # if the current word from the set is found in the context
    
    
    k=0
    for val in word_SET:
        for j in range(len(contex)):
            if(val in contex[j]):
                collocalationS[k,(int(clas[j])-1)]=collocalationS[k,(int(clas[j])-1)]+1 #MODIFIED to add-1 to index
        k=k+1
        
    #Returning the matrix and the set of word corresponding the rows in the matrix
    return word_SET,collocalationS

#testing the method

word_SET1,collocs1=collocalatsK(class1,context1[0:100])


def collocalatsPLUS1(clas, contex,word):
    
    #Making a one big string of all the context sentences
    wordS=" ".join(contex)
    #Splitting senteces into words
    wordS=wordS.split(' ')
    #Making a set of those words
    word_SET=set(wordS)
    
    classes=int(max(clas))
    #Making a matrix of the length of the words in a set
    #and a width of the different senses of the target word
    collocalationS=np.zeros((len(word_SET),classes)) #MODIFIED to exclude +1
    
    #In the 1. for loop going through of all the words in a set
    #In the 2. for loop going through of all the contexts and checking
    # if the current word from the set is found in the context
    
    
    k=0
    for val in word_SET:
        for j in range(len(contex)):
            val1=" ".join([val,word])
            if(val1 in contex[j]):
                collocalationS[k,(int(clas[j])-1)]=collocalationS[k,(int(clas[j])-1)]+1 #MODIFIED to add-1 to index
        k=k+1
        
    #Returning the matrix and the set of word corresponding the rows in the matrix
    return word_SET,collocalationS


def collocalatsMINUS1(clas, contex,word):
    
    #Making a one big string of all the context sentences
    wordS=" ".join(contex)
    #Splitting senteces into words
    wordS=wordS.split(' ')
    #Making a set of those words
    word_SET=set(wordS)
    
    classes=int(max(clas))
    #Making a matrix of the length of the words in a set
    #and a width of the different senses of the target word
    collocalationS=np.zeros((len(word_SET),classes)) #MODIFIED to exclude +1
    
    #In the 1. for loop going through of all the words in a set
    #In the 2. for loop going through of all the contexts and checking
    # if the current word from the set is found in the context
    
    
    k=0
    for val in word_SET:
        for j in range(len(contex)):
            val1=" ".join([word,val])
            if(val1 in contex[j]):
                collocalationS[k,(int(clas[j])-1)]=collocalationS[k,(int(clas[j])-1)]+1 #MODIFIED to add-1 to index
        k=k+1
        
    #Returning the matrix and the set of word corresponding the rows in the matrix
    return word_SET,collocalationS
#testing the method

word_SET1,collocs1=collocalatsK(class1,context1[0:80]) #MODIFIED here to use K collocation

#Printing all the words from the set which appear in more than x contexts
# and also printing the corresponding row of the Step 3 matrix

index=0
for val in word_SET1:
    if(np.sum(collocs1[index,:])>2):
        print(val,collocs1[index,:])
    index=index+1

comb1=np.column_stack((list(word_SET1), collocs1))
#print(comb1)

 [55. 25.]
into [1. 2.]
who [4. 0.]
singer [5. 0.]
with [6. 0.]
of [14.  4.]
music [8. 0.]
The [3. 1.]
instrumental [4. 0.]
star [5. 0.]
is [8. 3.]
all [1. 3.]
roll [10.  0.]
one [3. 1.]
be [10.  4.]
to [12.  9.]
and [9. 5.]
at [12. 10.]
a [48. 24.]
from [1. 2.]
in [20. 17.]
as [9. 1.]
throwing [0. 4.]
best [7. 0.]
out [4. 2.]
A [0. 3.]
concert [3. 0.]
the [20.  7.]
vocal [3. 0.]
ban [2. 1.]
architect [4. 0.]
rock [55. 25.]
n [50. 21.]
about [1. 2.]
he [27.  9.]
or [11.  7.]
hard [3. 0.]
group [5. 0.]
for [9. 3.]
create [0. 3.]
up [5. 1.]
on [13.  8.]
6 [1. 2.]
T [6. 2.]
an [19.  8.]
his [4. 2.]
legend [3. 0.]


In [17]:
#calculate logs
def logcalc(collocalationS):   
    logarray=np.zeros((collocalationS.shape[0],collocalationS.shape[1]))
    very_small_number = 0.0001
    k=0
    j=0
    for k in range(logarray.shape[0]):
        for j in range(logarray.shape[1]):
            if collocalationS[k,j]!=0:
                p1=collocalationS[k,j]/np.sum(collocalationS[k,:])
                p2=(np.sum(collocalationS[k,:])-collocalationS[k,j])/np.sum(collocalationS[k,:])
                logarray[k,j]=np.abs(np.log(p1/(p2 if p2!=0 else very_small_number)))
            j=j+1
        k=k+1
            
    #Returning the matrix and the set of word corresponding the rows in the matrix
    return logarray

#testing the method

w=logcalc(collocs1)
comb2=np.column_stack((list(word_SET1), w))
#print(comb2)


#then print the most likely collocation and its meaning
#prints one extra zero column on the left
index=0

a=np.array([])
b=np.array([])
c=np.array([])
for val in word_SET1:
    if(np.sum(collocs1[index,:])>2): 
        print(val,max(w[index,:]),"sense:",np.argmax(w[index,:])+1) 
        a=np.append(a,max(w[index,:]))
        b=np.append(b,np.argmax(w[index,:]))
        c=np.append(c,val)
    index=index+1

 0.7884573603642703 sense: 1
into 0.6931471805599453 sense: 1
who 9.210340371976184 sense: 1
singer 9.210340371976184 sense: 1
with 9.210340371976184 sense: 1
of 1.252762968495368 sense: 1
music 9.210340371976184 sense: 1
The 1.0986122886681098 sense: 1
instrumental 9.210340371976184 sense: 1
star 9.210340371976184 sense: 1
is 0.9808292530117264 sense: 2
all 1.0986122886681098 sense: 1
roll 9.210340371976184 sense: 1
one 1.0986122886681098 sense: 1
be 0.9162907318741551 sense: 1
to 0.2876820724517809 sense: 2
and 0.5877866649021191 sense: 1
at 0.1823215567939546 sense: 1
a 0.6931471805599453 sense: 1
from 0.6931471805599453 sense: 1
in 0.16251892949777494 sense: 1
as 2.1972245773362196 sense: 1
throwing 9.210340371976184 sense: 2
best 9.210340371976184 sense: 1
out 0.6931471805599453 sense: 1
A 9.210340371976184 sense: 2
concert 9.210340371976184 sense: 1
the 1.0498221244986778 sense: 2
vocal 9.210340371976184 sense: 1
ban 0.6931471805599453 sense: 1
architect 9.210340371976184 sense: 

In [23]:
def predicto(clasTrain,train,clasTest,test,word):
    
    #clasTrain and clasTest varaible contains the senses of the contexts in the train and test variables
    #train and test contains contexts of the target word
    # word contains the target word as it is as a string
    
    #the clasTest variable is not necessary for the method to work
    #we just calculate the accuracy with it
    
    #it can also be just a array as long as the clasTest variable and
    #the method works
    
    #the method 
    #RETURNS 
    #the test contexts and the predicted senses for the contexts
    
    #we change the word variable so it works better with the collocalats methods
    wordMethod=" " + word + " "
    
    #we also make a variable for the predicted classes for the test set
    PredclasTest=np.zeros(len(clasTest))
    
    #we create the word sets and the collocalation frequencies with the methods
    word_SET_P,collocplus1=collocalatsPLUS1(clasTrain,train,wordMethod)
    word_SET_M,collocminus1=collocalatsMINUS1(clasTrain,train,wordMethod)
    word_SET_K,collock=collocalatsK(clasTrain,train)
    
    #then we use the logcalc to get the logs of the collocalations
    Logs_P=logcalc(collocplus1)
    Logs_M=logcalc(collocminus1)
    Logs_K=logcalc(collock)
    
    
    #This could be done easier but now we make lists for each different way of making
    #collocalations where we save the usable collocalations (in wordS variables) the 
    # Logs in logS variables and the corresponding senses in senS variables
    
    #we now have 3 different ways to do collocalations which are plus1, minus1 and +-k (where k is now 3)
    P_logS=np.array([])
    P_senS=np.array([])
    P_wordS=np.array([])
    
    index=0
       
    #with these loops we get the biggest log of the corresponding sense in the collocalation
    
    for val in word_SET_P:
        if(np.sum(collocplus1[index,:])>1):
            P_logS=np.append(P_logS,max(Logs_P[index,:]))
            P_senS=np.append(P_senS,np.argmax(Logs_P[index,:])+1)
            P_wordS=np.append(P_wordS,val)
        index=index+1
        
        
    #same as above but with minus 1 collocalation  
    M_logS=np.array([])
    M_senS=np.array([])
    M_wordS=np.array([])
    
    index=0
    
    for val in word_SET_M:
        if(np.sum(collocminus1[index,:])>1):
            M_logS=np.append(M_logS,max(Logs_M[index,:]))
            M_senS=np.append(M_senS,np.argmax(Logs_M[index,:])+1)
            M_wordS=np.append(M_wordS,val)
        index=index+1
            
    #same as above but with +- k  collocalation
    
    K_logS=np.array([])
    K_senS=np.array([])
    K_wordS=np.array([])
    
    index=0
    
    for val in word_SET_K:
        if(np.sum(collock[index,:])>1):
            K_logS=np.append(K_logS,max(Logs_K[index,:]))
            K_senS=np.append(K_senS,np.argmax(Logs_K[index,:])+1)
            K_wordS=np.append(K_wordS,val)
        index=index+1
    
    #then we add all the same kind of variables in one variables to make the
    #decision list simpler
    A_logS=np.append(P_logS,M_logS)
    A_logS=np.append(A_logS,K_logS)
    
    A_senS=np.append(P_senS,M_senS)
    A_senS=np.append(A_senS,K_senS)
    
    A_wordS=np.append(P_wordS,M_wordS)
    A_wordS=np.append(A_wordS,K_wordS)
    
    #now we go through the test contexts
    for j in range(len(test)):
        
        #we take the context in the index j
        #split it by spaces
        #make a new variable 
        #where we only add actual words and not the empty spaces
        
        contex_sent=test[j]
        contex_sent=contex_sent.lower()
        contex_sent=contex_sent.split(' ')
        contex_sent2=[]
        
        # getting rid of the empty spaces in the context
        for wor in contex_sent:
            if (wor!=""):
                contex_sent2.append(wor)
        #print(contex_sent)        
        #print(contex_sent2)
        
        #then we make a decision list for the specific context
        DecList=np.zeros((2,len(contex_sent2)))   
        
        #test printing
        print(contex_sent2)
        #test printing
        
        #then we go through the words in the specific context
        
        #we first find the index of the target word so we can
        #give the right log values for each word
        #the +1 and -1 word get a unique log value
        #but all the words get also a +-k log value
        #for +1 and -1 word we check that is the +- k value bigger
        #and only save the bigger one with the corresponding log value
        
        TwordI=contex_sent2.index(word)
       
        
        for m in range(len(contex_sent2)):
                    
            #this is the case when we are not in the +1 -1 neigbourhood
            #of the target word
            
            #so now we only use the +- k collocalations
            if(m!=TwordI and m!=(TwordI-1) and m!=(TwordI+1)):
                ind=0
                
                #we go through the collocalations which have been obtained
                #with the +- k collocalation method
                
                #if there is a corresponding word we get the log
                #and the sense for the target word and save it in the DecList
                for wor in K_wordS:
                    if(contex_sent2[m]==wor):
                        DecList[0,m]=K_logS[ind]
                        DecList[1,m]=K_senS[ind]
                        
                    ind=ind+1
                    
            #this is the case when we are in the neighbourhood of the -1 word        
            elif(m==TwordI-1):
                k=0
                HelpVar=np.zeros((2,2))
                
                
                #same as above but now we also go through the minus1
                #collocalations
                
                #we save the values in the HelpVar and then take the log and sense
                #pair which has the bigger log value and save it in the DecList
                
                ind=0
                for wor in K_wordS:
                    if(contex_sent2[m]==wor):
                        HelpVar[0,k]=K_logS[ind]
                        HelpVar[1,k]=K_senS[ind]
                        k=k+1
                    ind=ind+1
                
                
                ind=0
                for wor in M_wordS:
                    if(contex_sent2[m]==wor):
                        HelpVar[0,k]=M_logS[ind]
                        HelpVar[1,k]=M_senS[ind]
                    ind=ind+1
                    
                DecList[0,m]=HelpVar[0,np.argmax(HelpVar[0,:])]
                DecList[1,m]=HelpVar[1,np.argmax(HelpVar[0,:])]
                
            #this is the case when we are in the neighbourhood of the +1 word     
            elif(m==TwordI+1):
                
                #same as above but now K and plus1 collocalations
                k=0
                HelpVar=np.zeros((2,2))
                
                ind=0
                for wor in K_wordS:
                    if(contex_sent2[m]==wor):
                        HelpVar[0,k]=K_logS[ind]
                        HelpVar[1,k]=K_senS[ind]
                        k=k+1
                    ind=ind+1
                
                
                ind=0
                for wor in P_wordS:
                    if(contex_sent2[m]==wor):
                        HelpVar[0,k]=P_logS[ind]
                        HelpVar[1,k]=P_senS[ind]
                    ind=ind+1
                    
                DecList[0,m]=HelpVar[0,np.argmax(HelpVar[0,:])]
                DecList[1,m]=HelpVar[1,np.argmax(HelpVar[0,:])]
                      
        #now we have the conext specific decision list and then we just
        #choose the sense with the biggest log value
        
        #to get the accuracy we can save the predicted classes in a variable
        print(DecList)
        
        print()
        print('In the sentence')
        print(test[j])
        print('the word ',word)
        print('is understood as a sense',DecList[1,np.argmax(DecList[0,:])])
        
        #saving the sense in the variable PredclasTest
        PredclasTest[j]=DecList[1,np.argmax(DecList[0,:])]
        
        
        if('rock' in word):
            print('0 not sure')
            print('1 means music rock')
            print('2 means material rock')
            
        if(word=='bass'):
            print('0 not sure')
            print('1 means bass instrument')
            print('2 means bass voice')
            print('3 means bass fish')
            
        if(word=='crane'):
            print('0 not sure')
            print('1 crane bird')
            print('2 crane machine')
            
        print()      
        
    accuracy=0
    for o in range(len(PredclasTest)):
        #print(PredclasTest[o])
        #print(clasTest[o])
        if(PredclasTest[o]==clasTest[o]):
            accuracy=accuracy+1
    print(accuracy)
    print('accuracy is: ',accuracy/(len(PredclasTest)))
    
    return test,PredclasTest
    
        
predicto(class1[0:80],context1[0:80],class1[80:100],context1[80:100],'rock')

['decades', 'ago', 'with', 'rock', 'music', 'clubs', 'quaint']
[[0.         0.         9.21034037 0.         9.21034037 0.
  0.        ]
 [0.         0.         1.         0.         1.         0.
  0.        ]]

In the sentence
decades ago with  rock  music clubs quaint
the word  rock
is understood as a sense 1.0
0 not sure
1 means music rock
2 means material rock

['have', 'helped', 'incite', 'rock', 'throwing', 'palestinians', 'in']
[[9.21034037 0.         0.         0.         9.21034037 0.
  0.16251893]
 [1.         0.         0.         0.         2.         0.
  1.        ]]

In the sentence
have helped incite  rock  throwing Palestinians in
the word  rock
is understood as a sense 1.0
0 not sure
1 means music rock
2 means material rock

['as', 'a', 'folk', 'rock', 'singer', 'with', 'the']
[[2.19722458 0.69314718 0.         0.         9.21034037 9.21034037
  1.04982212]
 [1.         1.         0.         0.         1.         1.
  2.        ]]

In the sentence
as a folk  rock  si

(array(['decades ago with  rock  music clubs quaint',
        'have helped incite  rock  throwing Palestinians in',
        'as a folk  rock  singer with the',
        'of the biggest  rock  stars of the',
        'album when the  rock  guitarist died in',
        'budget was the  rock  bottom that NASA',
        'chest with a  rock  during a demonstration',
        'more interested in  rock  music and his',
        'two three chord  rock  or your slow',
        'was no roots  rock  hotbed',
        'country music jazz  rock  n roll and',
        'he played with  rock  veterans the Grateful',
        'an ounce of  rock  cocaine',
        'Hueco Tanks to  rock  climbers ending a',
        'hammered into the  rock  to aid tricky',
        'destroyed ancient Indian  rock  paintings but said',
        'on the sheer  rock  faces',
        'embed bolts into  rock  to secure safety',
        'areas of sand  rock  and scrubland officials',
        'teams bored through  rock  to reach her'], dt

Unsupervised learning (find k-collocations where word matches word in our table, learn new words from there)

In [24]:
import pandas as pd

word_SET1,collocs1=collocalatsK(class1,context1[0:80]) #K collocation
#calculate logs
w=logcalc(collocs1)

#create a dataframe for decision list
columns = ['word','log', 'sense']
df = pd.DataFrame(columns=columns)
df = df.fillna(0) # with 0s rather than NaNs

def populatedecisionlist(decisionlist,word_set,collocs,logs):
    index=0
    for val in word_set:
        if(np.sum(collocs[index,:])>1): #may need to add >2? 
            if decisionlist[decisionlist.word ==val].shape[0]>=1: #if value exists in decision list, add log and calculate mean
                decisionlist['log'][decisionlist.word ==val]=(max(logs[index,:])+max(decisionlist['log'][decisionlist.word ==val]))/2
            #append word, max log, sense
            else:
                decisionlist = decisionlist.append(pd.Series([val,max(logs[index,:]),np.argmax(logs[index,:])+1], index=decisionlist.columns ), ignore_index=True)
        index=index+1
    return decisionlist

df=populatedecisionlist(df,word_SET1,collocs1,w)
print(df.sort_values(by=['log'],ascending=False))


       word       log sense
35  classic  9.210340     1
45  concert  9.210340     1
26     punk  9.210340     1
27   Grammy  9.210340     1
30    after  9.210340     2
..      ...       ...   ...
28       to  0.287682     2
31       at  0.182322     1
37       in  0.162519     1
36    broke  0.000000     1
24    which  0.000000     1

[71 rows x 3 columns]


In [25]:
def collocalats(contextnew,decisionlist): #create new senses
    learnedcontext=[]
    ind=0
    for collocation in contextnew:
        #Splitting senteces into words
        wordS=collocation.split(' ')
        #Making a set of those words
        word_SET=set(wordS)
        insidelist_log=[]
        insidelist_sense=[]
        for word in word_SET:
            if decisionlist[decisionlist.word ==word].shape[0]>=1: #if word is in the old table
                #get filtered dataframe that matches the word
                dffiltered=(decisionlist[decisionlist['word'] == word])
                insidelist_log.append(max(dffiltered['log'][dffiltered['log']==dffiltered['log'].max()]) ) #add log (from the biggest log)
                insidelist_sense.append(max(dffiltered['sense'][dffiltered['log']==dffiltered['log'].max()]) ) #add meaning (from the biggest log)
            else:
                insidelist_log.append(0)
                insidelist_sense.append(0)
                continue
        indexofmaxlog=insidelist_log.index((max(insidelist_log)))
        learnedcontext.append(insidelist_sense[indexofmaxlog]) #so that we have only one meaning per context before moving to new collocation
    return learnedcontext
 
contextnew=context1[100:200]    
cx=collocalats(contextnew,df)
print(cx)

[1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1]


Pass sense + context to the function;create a set of all the new words;calculate collocation from there

In [26]:
#repeat dataframe population for new learned words
#calculate logs
import numpy as np
word_SETnew,collocsnew=collocalatsK(cx,contextnew)

combnew=np.column_stack((list(word_SETnew), collocsnew))
wnew=logcalc(collocsnew)

dflearn=populatedecisionlist(df,word_SETnew,collocsnew,wnew)
print(dflearn.sort_values(by=['log'],ascending=False))


C:\Users\iced\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


          word      log sense
128       like  9.21034     1
45     concert  9.21034     1
74    musician  9.21034     1
73     Against  9.21034     2
72          22  9.21034     1
..         ...      ...   ...
81       their  0.00000     1
89   guitarist  0.00000     1
95      bottom  0.00000     1
24       which  0.00000     1
99        head  0.00000     1

[129 rows x 3 columns]


In [27]:
#classify 20 test instances using the new decision list table
contexttrain=context1[80:100]
cx=collocalats(contexttrain,dflearn) #learn new classifications

word_SETtest,collocstest=collocalatsK(cx,contexttrain)

clasTest=class1[80:100]
accuracy=0
for o in range(len(cx)):
    if(cx[o]==clasTest[o]):
        accuracy=accuracy+1
print('accuracy is: ',accuracy/(len(cx)))

accuracy is:  0.55


In [28]:
#Example 2 - bass
#extract all sentences containing 'bass '
examples2=[sentence for sentence in sentencecleaned if ' bass ' in sentence]
examples2=examples2[0:200] #subsetting

%store examples2
#%store -r examples2

#print(examples2) #100 whole sentences

Stored 'examples2' (list)


In [29]:
context2=[]
context2=contexts(examples2,context2,' bass ')

In [30]:
import numpy as np

class2 = np.array([1,1,1,1,1,1,1,1,3,3,2,1,2,2,2,2,1,1,1,1,1,2,1,1,2,3,1,3,2,2,1,2,1,3,1,1,3,3,3,3,3,3,1,1,2,1,1,1,2,1,1,2,2,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,1,1,1,3,3,3,1,3,3,3,1,2,2,2,1,1,3,3,1,1,0,3,2,2,1,1,1,1,2,1,2,2])
shape = (10,10)
class2.reshape( shape )

array([[1, 1, 1, 1, 1, 1, 1, 1, 3, 3],
       [2, 1, 2, 2, 2, 2, 1, 1, 1, 1],
       [1, 2, 1, 1, 2, 3, 1, 3, 2, 2],
       [1, 2, 1, 3, 1, 1, 3, 3, 3, 3],
       [3, 3, 1, 1, 2, 1, 1, 1, 2, 1],
       [1, 2, 2, 1, 1, 1, 3, 3, 3, 3],
       [3, 3, 3, 3, 3, 3, 3, 3, 1, 1],
       [1, 3, 3, 3, 1, 3, 3, 3, 1, 2],
       [2, 2, 1, 1, 3, 3, 1, 1, 0, 3],
       [2, 2, 1, 1, 1, 1, 2, 1, 2, 2]])

In [31]:
predicto(class2[0:80],context2[0:80],class2[80:100],context2[80:100],'bass')

['some', 'fret', 'popping', 'bass', 'from', 'william', 'bootsy']
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]]

In the sentence
some fret popping  bass  from William Bootsy
the word  bass
is understood as a sense 0.0
0 not sure
1 means bass instrument
2 means bass voice
3 means bass fish

['but', 'charismatic', 'bass', 'paata', 'burchuladze', 'as']
[[9.21034037 0.         0.         0.         0.         1.46633707]
 [3.         0.         0.         0.         0.         2.        ]]

In the sentence
But charismatic  bass  Paata Burchuladze as
the word  bass
is understood as a sense 3.0
0 not sure
1 means bass instrument
2 means bass voice
3 means bass fish

['the', 'double', 'reed', 'bass', 'woodwind', 'for', '25']
[[1.46633707 0.         0.         0.         0.         0.69314718
  0.        ]
 [2.         0.         0.         0.         0.         1.
  0.        ]]

In the sentence
the double reed  bass  woodwind for 25
the word  bass
is understood as a sense 2.0
0 not sure
1 

(['some fret popping  bass  from William Bootsy',
  'But charismatic  bass  Paata Burchuladze as',
  'the double reed  bass  woodwind for 25',
  'rubber hoses on  bass  drums',
  'a white sea  bass  at a wholesale',
  'shrimp oyster striped  bass  and redfish industies',
  'write the songs  bass  guitar player John',
  'of a 1965  bass  drum bearing the',
  'on a monthly  bass  but also trade',
  'gamefish such as  bass  or perch from',
  '6 foot 3  bass  baritone will sing',
  'Sterban the booming  bass  of the Oak',
  'Another highlight was  bass  player Phil Lesh',
  'and Oates and  bass  player Jack Casady',
  'the vocalist a  bass  guitarist and himself',
  'singer John Wiegand  bass  player Rob Wilhelm',
  'role so that  bass  baritone Norman Treigle',
  'Ron Brooks a  bass  player and owner',
  'Rumanian born  bass  Julian Rodescu now',
  'Placenza DAdige Italy  bass  Janusz Borowicz Krakow'],
 array([0., 3., 2., 1., 2., 3., 1., 1., 3., 2., 2., 2., 1., 1., 1., 2., 2.,
        1.

In [32]:
#unsupervised check
#populate initial decision list
word_SET2,collocs2=collocalatsK(class2,context2[0:80]) #K collocation
#calculate logs
w2=logcalc(collocs2)

#create a dataframe for decision list
columns = ['word','log', 'sense']
df2 = pd.DataFrame(columns=columns)
df2 = df2.fillna(0) # with 0s rather than NaNs

df2=populatedecisionlist(df2,word_SET2,collocs2,w2)
print(df2.sort_values(by=['log'],ascending=False))

        word      log sense
72      like  9.21034     1
32    Ronnie  9.21034     1
47  baritone  9.21034     2
24   striped  9.21034     3
25     Cheek  9.21034     2
..       ...      ...   ...
55      John  0.00000     1
45       was  0.00000     1
51       man  0.00000     1
7        got  0.00000     1
23        us  0.00000     1

[73 rows x 3 columns]


In [33]:
#populate decision list with new data
contextnew2=context2[100:200]    
cx2=collocalats(contextnew2,df2)

word_SETnew2,collocsnew2=collocalatsK(cx2,contextnew2)

combnew=np.column_stack((list(word_SETnew2), collocsnew2))
wnew2=logcalc(collocsnew2)

dflearn2=populatedecisionlist(df2,word_SETnew2,collocsnew2,wnew2)
print(dflearn2.sort_values(by=['log'],ascending=False))

C:\Users\iced\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


           word      log sense
60    guitarist  9.21034     1
89          put  9.21034     2
85         deep  9.21034     2
83      Society  9.21034     3
82     concerto  9.21034     2
..          ...      ...   ...
7           got  0.00000     1
97          her  0.00000     1
98      private  0.00000     1
99        hours  0.00000     1
79  performance  0.00000     1

[121 rows x 3 columns]


In [34]:
#classify 20 test instances using the new decision list table
contexttrain2=context2[80:100]
cx2=collocalats(contexttrain2,dflearn2) #learn new classifications

word_SETtest2,collocstest2=collocalatsK(cx2,contexttrain2)

clasTest2=class2[80:100]
accuracy2=0
for o in range(len(cx2)):
    if(cx2[o]==clasTest2[o]):
        accuracy2=accuracy2+1
print('accuracy is: ',accuracy2/(len(cx2)))

accuracy is:  0.75


In [35]:
#Example 3 - crane
examples3=[sentence for sentence in sentencecleaned if ' crane ' in sentence]
examples3=examples3[0:200] #subsetting

%store examples3
#%store -r examples3

Stored 'examples3' (list)


In [36]:
context3=[]
context3=contexts(examples3,context3,' crane ')

In [37]:
#
# IMPORTANT!!!!
#
# only run the cells which use the delete method ONLY ONCE
# 
#


#the below one has 101 classes because of the one verb in the midst which is deleted in this cell too
class3=np.array([2,2,2,2,2,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,1,1,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2])
class3=np.delete(class3,27)
print(class3)
#shape = (10,10)
#class3.reshape( shape )
context3=np.delete(context3,27)

[2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 2 1 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1
 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [38]:
predicto(class3[0:80],context3[0:80],class3[80:100],context3[80:100],'crane')

['a', 'single', 'whooping', 'crane', 'lives', 'in', 'a']
[[1.73460106 0.         9.21034037 0.         0.         1.17007125
  1.73460106]
 [1.         0.         1.         0.         0.         1.
  1.        ]]

In the sentence
a single whooping  crane  lives in a
the word  crane
is understood as a sense 1.0
0 not sure
1 crane bird
2 crane machine

['it', 'on', 'a', 'crane', 'and', 'it', 'scaled']
[[0.         1.84582669 9.21034037 0.         1.84582669 0.
  0.        ]
 [0.         1.         2.         0.         1.         0.
  0.        ]]

In the sentence
it on a  crane  and it scaled
the word  crane
is understood as a sense 2.0
0 not sure
1 crane bird
2 crane machine

['28', 'year', 'old', 'crane', 'driver', 'fled', 'over']
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]

In the sentence
28 year old  crane  driver fled over
the word  crane
is understood as a sense 0.0
0 not sure
1 crane bird
2 crane machine

['helgoland', 'by', 'a', 'crane', 'as', 'performer', 'dirk']
[[0.   

(array(['a single whooping  crane  lives in a',
        'it on a  crane  and it scaled',
        '28 year old  crane  driver fled over',
        'Helgoland by a  crane  as performer Dirk',
        'when a construction  crane  began swaying as',
        'lowered and the  crane  secured a short',
        'A city  crane  helped workers take',
        'AP A floating  crane  on Tuesday raised',
        'A  crane  was brought in',
        'controls of a  crane  and gleefully wields',
        'worked as a  crane  operator',
        'Krystana Burant a  crane  operator who has',
        'was removed by  crane  and taken away',
        'last month a  crane  operator suffocated after',
        'company has given  crane  operators instructions on',
        'week on a  crane  next to a',
        'even when a  crane  lowers a Volvo',
        'hoisted up by  crane  to the billboard',
        'A  crane  was brought in', 'A  crane  was used to'], dtype='<U54'),
 array([1., 2., 0., 2., 2., 1., 1., 2., 2

In [39]:
#unsupervised check
#populate initial decision list
word_SET3,collocs3=collocalatsK(class3,context3[0:80]) #K collocation
#calculate logs
w3=logcalc(collocs3)

#create a dataframe for decision list
columns = ['word','log', 'sense']
df3 = pd.DataFrame(columns=columns)
df3 = df3.fillna(0) # with 0s rather than NaNs

df3=populatedecisionlist(df3,word_SET3,collocs3,w3)
print(df3.sort_values(by=['log'],ascending=False))

       word       log sense
72     were  9.210340     2
26    giant  9.210340     2
28       AP  9.210340     2
30  habitat  9.210340     1
32  species  9.210340     1
..      ...       ...   ...
15     that  0.693147     1
38     from  0.693147     1
16     them  0.000000     1
9       The  0.000000     1
23    would  0.000000     1

[73 rows x 3 columns]


In [40]:
#populate decision list with new data
contextnew3=context3[100:200]    
cx3=collocalats(contextnew3,df3)

word_SETnew3,collocsnew3=collocalatsK(cx3,contextnew3)

combnew=np.column_stack((list(word_SETnew3), collocsnew3))
wnew3=logcalc(collocsnew3)

dflearn3=populatedecisionlist(df3,word_SETnew3,collocsnew3,wnew3)
print(dflearn3.sort_values(by=['log'],ascending=False))

       word      log sense
94    three  9.21034     1
30  habitat  9.21034     1
32  species  9.21034     1
67     said  9.21034     2
34      has  9.21034     2
..      ...      ...   ...
89  control  0.00000     1
9       The  0.00000     1
92  arrived  0.00000     1
93       up  0.00000     1
73  lowered  0.00000     1

[95 rows x 3 columns]


C:\Users\iced\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [41]:
#classify 20 test instances using the new decision list table
contexttrain3=context3[80:100]
cx3=collocalats(contexttrain3,dflearn3) #learn new classifications

word_SETtest3,collocstest3=collocalatsK(cx3,contexttrain3)

clasTest3=class3[80:100]
accuracy3=0
for o in range(len(cx3)):
    if(cx3[o]==clasTest3[o]):
        accuracy3=accuracy3+1
print('accuracy is: ',accuracy3/(len(cx3)))

accuracy is:  0.75
